In [1]:
from collections import Counter
import random
from statistics import variance
from typing import List, Set, Tuple, Union

In [2]:

class Match:  # Gene
    def __init__(self, team_1: Union[List, Set, Tuple], team_2: Union[List, Set, Tuple]):
        self.team_1 = set(team_1)
        self.team_2 = set(team_2)
        assert not self.team_1.intersection(self.team_2), "No overlapping players between teams"

    def __eq__(self, other):
        return ((self.team_1 == other.team_1) and (self.team_2 == other.team_2)
                or (self.team_1 == other.team_2) and (self.team_2 == other.team_1))

    def __hash__(self):
        return hash(f"{sorted(self.team_1)}.{sorted(self.team_2)}")

    def __str__(self):
        return f'{self.team_1} - {self.team_2}'

    def to_list(self):
        return list(self.team_1) + list(self.team_2)


# For simplicity, let's start with the case of a single Gene (match) played
# simultaneously.

class Tournament:  # Genome
    def __init__(self, num_players, num_matches, matches: List['Match'] = None):
        self.num_players = num_players
        self.num_matches = num_matches
        if not matches:
            self.matches = [self._make_random_match() for _ in range(self.num_matches)]
        else:
            self.matches = matches

    @property
    def fitness(self):
        # number of unique matches
        unique_matches = set()
        players = []
        for g in self.matches:
            unique_matches.add(g)
            players.extend(g.to_list())
        return len(unique_matches) * (1 - variance(Counter(players).values()))

    def _make_random_match(self):
        num_players_for_match = 4
        players = random.sample(range(self.num_players), num_players_for_match)
        return Match(players[:num_players_for_match // 2], players[num_players_for_match // 2:])

    def mutate(self):
        for i in range(len(self.matches)):
            if random.random() < 0.9:
                # TODO: do I want mutated gene to be based on original?
                self.matches[i] = self._make_random_match()

    def __str__(self):
        return '\n'.join([str(g) for g in self.matches])


def crossover(a: 'Tournament', b: 'Tournament'):
    offspring_matches = []
    for left, right in zip(a.matches, b.matches):
        if random.random() < 0.5:
            offspring_matches.append(left)
        else:
            offspring_matches.append(right)
    return Tournament(a.num_players, a.num_matches, offspring_matches)


def initial_population(size, num_players, num_matches):
    population = []
    for _ in range(size):
        population.append(Tournament(num_players, num_matches))
    return population


def next_generation(current_generation):
    tournaments_by_fitness = sorted(current_generation, key=lambda g: g.fitness)
    # fittest from current generation will survive unmutated into future generation
    elite = tournaments_by_fitness[-1]
    # all genomes from current generation that are in the top 40% based on fitness will reproduce
    breeding_population = set(tournaments_by_fitness[int(len(current_generation) * 0.6):])

    children = [elite]
    while len(children) < len(current_generation):
        a = random.choice(list(breeding_population))
        b = random.choice(list(breeding_population - {a}))
        child = crossover(a, b)
        child.mutate()
        children.append(child)

    return children


In [5]:
%%time
num_players = 12
num_matches = 10
population = initial_population(100, num_players, num_matches)

elite = sorted(population, key=lambda g: g.fitness)[-1]
generation_elite_found = 0

for i in range(1000):  # num generations
    population = next_generation(population)
    elite_in_generation = sorted(population, key=lambda g: g.fitness)[-1]
    if elite_in_generation.fitness > elite.fitness:
        elite = elite_in_generation
        generation_elite_found = i

print(generation_elite_found)
print(elite)



11
{6, 7} - {0, 3}
{1, 11} - {10, 6}
{10, 2} - {0, 4}
{3, 4} - {0, 6}
{1, 10} - {5, 7}
{8, 11} - {9, 2}
{1, 2} - {3, 7}
{9, 2} - {3, 7}
{8, 10} - {4, 5}
{11, 5} - {8, 9}
Wall time: 27.6 s


In [6]:
players = []
for g in elite.matches:
    players.extend(g.to_list())
print(Counter(players))
    

Counter({7: 4, 3: 4, 10: 4, 2: 4, 6: 3, 0: 3, 1: 3, 11: 3, 4: 3, 5: 3, 8: 3, 9: 3})
